#### Notebook for data exporation, cleaning and preparation. To ensure consistent and ready to query data is loaded to database.

## Read dataset

In [1]:
import sys
import re
import pandas as pd
import numpy as np
from ast import literal_eval
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 30)


In [2]:
df = pd.read_csv('docs/books_1.Best_Books_Ever.csv')
df.head(2)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,bookFormat,edition,pages,publisher,publishDate,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FOR...,English,9780439023481,"['Young Adult', 'Fiction',...","['Katniss Everdeen', 'Peet...",Hardcover,First Edition,374,Scholastic Press,09/14/08,NaN,['Locus Award Nominee for ...,6376780,"['3444695', '1921313', '74...",96.0,"['District 12, Panem', 'Ca...",https://i.gr-assets.com/im...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Ord...,Harry Potter and the Order...,Harry Potter #5,"J.K. Rowling, Mary GrandPr...",4.50,There is a door at the end...,English,9780439358071,"['Fantasy', 'Young Adult',...","['Sirius Black', 'Draco Ma...",Paperback,US Edition,870,Scholastic Inc.,09/28/04,06/21/03,['Bram Stoker Award for Wo...,2507623,"['1593642', '637516', '222...",98.0,['Hogwarts School of Witch...,https://i.gr-assets.com/im...,2632233,26923,7.38


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52478 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52478 non-null  object 
 1   title             52478 non-null  object 
 2   series            23470 non-null  object 
 3   author            52478 non-null  object 
 4   rating            52478 non-null  float64
 5   description       51140 non-null  object 
 6   language          48672 non-null  object 
 7   isbn              52478 non-null  object 
 8   genres            52478 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        51005 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             50131 non-null  object 
 13  publisher         48782 non-null  object 
 14  publishDate       51598 non-null  object 
 15  firstPublishDate  31152 non-null  object 
 16  awards            52478 non-null  object

## Dataset cleaning 

In [4]:
# Remove duplicates
print('Number of rows before: ', df['bookId'].size)
df0 = df.drop_duplicates()
print('Number of rows after: ', df0['bookId'].size)


Number of rows before:  52478
Number of rows after:  52428


### Verify `bookId` uniquness and clean `price`

In [5]:
# Update price format. From string like '1.189.88' to float 1189.88
df01 = df0.copy()
pat = r'\.(\d{3})'
repl = lambda m: m.group().replace(".", "")
df01['price'] = df01['price'].str.replace(pat, repl, regex=True)
df01['price'] = df01['price'].astype('float')
#df01[df01['price']=='1.56.91']
df01['price']

0        5.09
1        7.38
2         NaN
3         NaN
4        2.10
         ... 
52473     NaN
52474     NaN
52475    7.37
52476    2.86
52477    5.20
Name: price, Length: 52428, dtype: float64

In [6]:
# check if bookId is unique.
df01.groupby('bookId')['bookId'].count().sort_values(ascending=False)

bookId
24903989-widz-ci                            2
635270.The_Planet_Pirates                   2
975953.Time_of_the_Dragons                  2
60614.Diamond_Dogs                          2
1.Harry_Potter_and_the_Half_Blood_Prince    1
                                           ..
19271017-my-story                           1
1927111.Love_Com_Vol_5                      1
1927112.Love_Com_Vol_6                      1
192722.Candle_in_the_Darkness               1
999985.Horrid_Henry_s_Underpants            1
Name: bookId, Length: 52424, dtype: int64

In [7]:
# Above related issue is with 'price' column. 2 slightly different prices for the same bookId
df01[df01['bookId'].isin(['60614.Diamond_Dogs',
                        '24903989-widz-ci',
                        '635270.The_Planet_Pirates',
                        '975953.Time_of_the_Dragons'])].sort_values('bookId')

,bookId,title,series,author,rating,description,language,isbn,genres,characters,bookFormat,edition,pages,publisher,publishDate,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
37422,24903989-widz-ci,Widzę cię,Trilogia dei sensi #1,Irene Cao,3.22,Gdyby dało się uchwycić pr...,Polish,9788379991587,"['Romance', 'Erotica', 'Co...","['Elena Kyler', 'Leonardo']",Paperback,NaN,304,Sonia Draga,January 14th 2015,11/06/13,[],1292,"['216', '321', '405', '235...",73.0,['Venice (Italy)'],https://i.gr-assets.com/im...,87,1,7.28
37476,24903989-widz-ci,Widzę cię,Trilogia dei sensi #1,Irene Cao,3.22,Gdyby dało się uchwycić pr...,Polish,9788379991587,"['Romance', 'Erotica', 'Co...","['Elena Kyler', 'Leonardo']",Paperback,NaN,304,Sonia Draga,January 14th 2015,11/06/13,[],1292,"['216', '321', '405', '235...",73.0,['Venice (Italy)'],https://i.gr-assets.com/im...,87,1,7.32
37401,60614.Diamond_Dogs,Diamond Dogs,NaN,Alan Watt (Goodreads Author),3.46,Neil Garvin is a seventeen...,English,9780446677844,"['Fiction', 'Mystery', 'Co...",[],Paperback,NaN,256,Grand Central Publishing,September 1st 2001,09/01/00,['ALA Alex Award (2001)'],320,"['54', '100', '115', '42',...",84.0,[],https://i.gr-assets.com/im...,87,1,6.27
37455,60614.Diamond_Dogs,Diamond Dogs,NaN,Alan Watt (Goodreads Author),3.46,Neil Garvin is a seventeen...,English,9780446677844,"['Fiction', 'Mystery', 'Co...",[],Paperback,NaN,256,Grand Central Publishing,September 1st 2001,09/01/00,['ALA Alex Award (2001)'],320,"['54', '100', '115', '42',...",84.0,[],https://i.gr-assets.com/im...,87,1,6.06
37399,635270.The_Planet_Pirates,The Planet Pirates,Planet Pirates #1-3,"Anne McCaffrey, Elizabeth ...",4.09,THE PLANET PIRATES ENSLAVE...,English,9780671319625,"['Science Fiction', 'Fanta...","['Sassinak', 'Lunzie']",Hardcover,NaN,890,Baen Books,December 1st 2000,11/06/93,[],1287,"['515', '444', '269', '49'...",95.0,[],https://i.gr-assets.com/im...,87,2,7.72
37453,635270.The_Planet_Pirates,The Planet Pirates,Planet Pirates #1-3,"Anne McCaffrey, Elizabeth ...",4.09,THE PLANET PIRATES ENSLAVE...,English,9780671319625,"['Science Fiction', 'Fanta...","['Sassinak', 'Lunzie']",Hardcover,NaN,890,Baen Books,December 1st 2000,11/06/93,[],1287,"['515', '444', '269', '49'...",95.0,[],https://i.gr-assets.com/im...,87,2,7.74
37423,975953.Time_of_the_Dragons,Time of the Dragons,Shike #1,Robert Shea,4.27,Shike is a novel about two...,English,9780515048742,"['Historical Fiction', 'Fi...",[],Paperback,NaN,453,Jove,June 1st 1981,NaN,[],496,"['237', '179', '62', '14',...",96.0,[],https://i.gr-assets.com/im...,87,1,5.45
37477,975953.Time_of_the_Dragons,Time of the Dragons,Shike #1,Robert Shea,4.27,Shike is a novel about two...,English,9780515048742,"['Historical Fiction', 'Fi...",[],Paperback,NaN,453,Jove,June 1st 1981,NaN,[],496,"['237', '179', '62', '14',...",96.0,[],https://i.gr-assets.com/im...,87,1,5.46


In [8]:
# Change these values to mean
df01['price'] = df01.groupby('isbn')['price'].transform('mean').round(2)

In [9]:
df01[df01['bookId'].isin(['60614.Diamond_Dogs',
                        '24903989-widz-ci',
                        '635270.The_Planet_Pirates',
                        '975953.Time_of_the_Dragons'])].sort_values('bookId')

,bookId,title,series,author,rating,description,language,isbn,genres,characters,bookFormat,edition,pages,publisher,publishDate,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
37422,24903989-widz-ci,Widzę cię,Trilogia dei sensi #1,Irene Cao,3.22,Gdyby dało się uchwycić pr...,Polish,9788379991587,"['Romance', 'Erotica', 'Co...","['Elena Kyler', 'Leonardo']",Paperback,NaN,304,Sonia Draga,January 14th 2015,11/06/13,[],1292,"['216', '321', '405', '235...",73.0,['Venice (Italy)'],https://i.gr-assets.com/im...,87,1,7.30
37476,24903989-widz-ci,Widzę cię,Trilogia dei sensi #1,Irene Cao,3.22,Gdyby dało się uchwycić pr...,Polish,9788379991587,"['Romance', 'Erotica', 'Co...","['Elena Kyler', 'Leonardo']",Paperback,NaN,304,Sonia Draga,January 14th 2015,11/06/13,[],1292,"['216', '321', '405', '235...",73.0,['Venice (Italy)'],https://i.gr-assets.com/im...,87,1,7.30
37401,60614.Diamond_Dogs,Diamond Dogs,NaN,Alan Watt (Goodreads Author),3.46,Neil Garvin is a seventeen...,English,9780446677844,"['Fiction', 'Mystery', 'Co...",[],Paperback,NaN,256,Grand Central Publishing,September 1st 2001,09/01/00,['ALA Alex Award (2001)'],320,"['54', '100', '115', '42',...",84.0,[],https://i.gr-assets.com/im...,87,1,6.16
37455,60614.Diamond_Dogs,Diamond Dogs,NaN,Alan Watt (Goodreads Author),3.46,Neil Garvin is a seventeen...,English,9780446677844,"['Fiction', 'Mystery', 'Co...",[],Paperback,NaN,256,Grand Central Publishing,September 1st 2001,09/01/00,['ALA Alex Award (2001)'],320,"['54', '100', '115', '42',...",84.0,[],https://i.gr-assets.com/im...,87,1,6.16
37399,635270.The_Planet_Pirates,The Planet Pirates,Planet Pirates #1-3,"Anne McCaffrey, Elizabeth ...",4.09,THE PLANET PIRATES ENSLAVE...,English,9780671319625,"['Science Fiction', 'Fanta...","['Sassinak', 'Lunzie']",Hardcover,NaN,890,Baen Books,December 1st 2000,11/06/93,[],1287,"['515', '444', '269', '49'...",95.0,[],https://i.gr-assets.com/im...,87,2,7.73
37453,635270.The_Planet_Pirates,The Planet Pirates,Planet Pirates #1-3,"Anne McCaffrey, Elizabeth ...",4.09,THE PLANET PIRATES ENSLAVE...,English,9780671319625,"['Science Fiction', 'Fanta...","['Sassinak', 'Lunzie']",Hardcover,NaN,890,Baen Books,December 1st 2000,11/06/93,[],1287,"['515', '444', '269', '49'...",95.0,[],https://i.gr-assets.com/im...,87,2,7.73
37423,975953.Time_of_the_Dragons,Time of the Dragons,Shike #1,Robert Shea,4.27,Shike is a novel about two...,English,9780515048742,"['Historical Fiction', 'Fi...",[],Paperback,NaN,453,Jove,June 1st 1981,NaN,[],496,"['237', '179', '62', '14',...",96.0,[],https://i.gr-assets.com/im...,87,1,5.46
37477,975953.Time_of_the_Dragons,Time of the Dragons,Shike #1,Robert Shea,4.27,Shike is a novel about two...,English,9780515048742,"['Historical Fiction', 'Fi...",[],Paperback,NaN,453,Jove,June 1st 1981,NaN,[],496,"['237', '179', '62', '14',...",96.0,[],https://i.gr-assets.com/im...,87,1,5.46


In [10]:
# Different price values for the same publication were susbtituted by mean,
# now it is deduplicated as there should be one price per publication. (-4 lines)

fix_col = list(df0.columns)
fix_col.remove('price')

df01 = df01.drop_duplicates(fix_col)

# problem solved
df01.groupby('bookId')['bookId'].count().sort_values(ascending=False)

bookId
1.Harry_Potter_and_the_Half_Blood_Prince    1
4068710-ufo-ifo                             1
40670008-before-the-fall                    1
40670312-the-one                            1
4067103-ransom-my-heart                     1
                                           ..
19271017-my-story                           1
1927111.Love_Com_Vol_5                      1
1927112.Love_Com_Vol_6                      1
192722.Candle_in_the_Darkness               1
999985.Horrid_Henry_s_Underpants            1
Name: bookId, Length: 52424, dtype: int64

In [11]:
df01.shape

(52424, 25)

### Clean `isbn`

In [12]:
# Issue many rows with "isbn" = 9999999999999
df01.groupby('isbn')['isbn'].count().sort_values(ascending=False).head(5)

isbn
9999999999999    4350
9781250166548       2
9780765326355       2
9780312429980       2
0000195166000       1
Name: isbn, dtype: int64

In [13]:
# book usually has a unique ISBN code. In order to use it as part of primary key for the book table
# the rows containing '9999999999999' will be substituted with numeric part of 'bookId'
df02 = df01.copy()
df02['isbn_new'] = df01['bookId'].str.extract(r'^(\d+)')
df02['isbn'] = np.where(df02['isbn'] =='9999999999999', df02['isbn_new'], df02['isbn'])
df03 = df02.drop(columns=['isbn_new'])

print('Number of unique ISBN: ', df03['isbn'].nunique())
print('Number of unique "bookId": ', df03['bookId'].nunique())

Number of unique ISBN:  52421
Number of unique "bookId":  52424


### The ISBN identifies not only the particular publication but its publisher. If there is a change of publisher, then the new publisher must assign one of their own ISBNs to the new publication.

In [14]:
# There are 1 incorrect line
print(df03.groupby(['isbn'])['publisher'].nunique().sort_values(ascending=False).head(5))
df03[df03['isbn'] == '9780312429980'].sort_values('title').head(3)

isbn
9780312429980    2
0000195166000    1
9781423143543    1
9781423144335    1
9781423145509    1
Name: publisher, dtype: int64


,bookId,title,series,author,rating,description,language,isbn,genres,characters,bookFormat,edition,pages,publisher,publishDate,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
1296,7826803-wolf-hall,Wolf Hall,Thomas Cromwell #1,Hilary Mantel (Goodreads A...,3.88,England in the 1520s is a ...,English,9780312429980,"['Historical Fiction', 'Fi...","['Anne Boleyn', 'Thomas Mo...",Paperback,NaN,604,Picador USA,August 31st 2010,04/30/09,"['Booker Prize (2009)', 'O...",166123,"['62692', '51592', '29341'...",86.0,['Putney (United Kingdom)'...,https://i.gr-assets.com/im...,4109,57,4.2
25628,19380923-wolf-hall,Wolf Hall,Thomas Cromwell #1,Hilary Mantel (Goodreads A...,3.88,Tudor England. Henry VIII ...,English,9780312429980,"['Historical Fiction', 'Fi...","['Anne Boleyn', 'Thomas Mo...",Kindle Edition,NaN,672,Fourth Estate,January 16th 2010,04/30/09,"['Booker Prize (2009)', 'O...",166308,"['62767', '51656', '29360'...",86.0,['Putney (United Kingdom)'...,https://i.gr-assets.com/im...,98,1,4.2


In [15]:
# drop the line with less info
print('Lines before: ', df03.shape[0])
df04 = df03.loc[~((df03["isbn"] == '9780312429980') & (df03["bookFormat"] == 'Kindle Edition'))]
print('Lines after: ', df04.shape[0])

Lines before:  52424
Lines after:  52423


### The same ISBN cannot be used for different formats of the book, whether the book is in printed or electronic form. That means hardcover and softcover versions of the book have to be assigned separate ISBNs.

In [16]:
df04.groupby(['isbn'])['bookFormat'].nunique().sort_values(ascending=False).head(5)

isbn
9780765326355    2
9781250166548    2
9781452373218    1
9781452303680    1
9781452305080    1
Name: bookFormat, dtype: int64

In [17]:
# Update isbn with digits from bookId to make it unique
df05 = df04.copy()

def replace_isbn(row):
    if row['isbn'] in ('9781250166548', '9780765326355'):
        m = re.search(r'^\d+', row['bookId'])
        return m.group()
    else:
        return row['isbn']
    
df05['isbn'] = df05.apply(replace_isbn, axis=1)
df05[df05['isbn']=='9781250166548']

df05.groupby(['isbn'])['bookFormat'].nunique().sort_values(ascending=False).head(5)

isbn
0000195166000    1
9781452365862    1
9781452300795    1
9781452301679    1
9781452303680    1
Name: bookFormat, dtype: int64

### Clean `description`

In [18]:
# '\"' found in one book description is impending load as it is not escaped.
df05['description'] = df05['description'].str.replace(r'\\"', '', regex=True)

In [19]:
pd.set_option('display.max_colwidth', 50)
df05.groupby(['isbn'])['description'].nunique().sort_values(ascending=False).head(4)

isbn
0000195166000    1
9781452082448    1
9781451695199    1
9781451695656    1
Name: description, dtype: int64

### Clean `publishDate` and `firstPublishDate`

In [20]:
df05.shape

(52423, 25)

In [21]:
# too long values for date `publishDate`
print('max length for publishDate: ', df05['publishDate'].str.len().max())
print('max length for firstPublishDate: ',df05['firstPublishDate'].str.len().max())

max length for publishDate:  209.0
max length for firstPublishDate:  19.0


In [22]:
# the look of this long date
pd.set_option('display.max_colwidth', 300)
df05[df05['publishDate'].str.len()==209]['publishDate']

39175    Books With a Goodreads Average Rating of 4.5 or higher and With At Least 100 Ratings\r\n\r\n578 books — 3,595 voters\r\nBooks with a Goodreads score of 4.4 or above with at least 300 ratings\r\n\r\n254 books — 19 voters
Name: publishDate, dtype: object

In [23]:
# max sensible data string length including month names is 20.
# But there are over 300 longer entries to data column.

print(" Quantity of lines for 'publishDate' with length over 20: ", df05[(df05['publishDate'].str.len() > 20)]['publishDate'].count())
df05[df05['publishDate'].str.len() > 20].publishDate.head(2)


 Quantity of lines for 'publishDate' with length over 20:  304


2989    Best Books to Read When the Snow Is Falling\r\n\r\n3,839 books — 3,426 voters\r\nI Had No Idea This Book Would Be So Good\r\n\r\n3,520 books — 2,105 voters
3560                                    Most Interesting World\r\n\r\n3,055 books — 2,474 voters\r\nThe Best Omnibuses and Box Sets\r\n\r\n474 books — 1,604 voters
Name: publishDate, dtype: object

In [24]:
# Clean not date data format.  Change to null.
df06 = df05.copy()
print('Nulls before: ', df06['publishDate'].isna().sum())

df06['publishDate'] = df06['publishDate'].str.strip()
df06['publishDate'] = np.where(df06['publishDate'].str.len() > 20, np.NaN, df06['publishDate'])
df06['publishDate'] = np.where(df06['publishDate'].str.contains(
    r'^[a-zA-Z]*$|^([a-zA-Z]*\s)+[a-zA-Z]*$', na=False, regex=True), np.NaN, df06['publishDate'])

print('Nulls after: ', df06['publishDate'].isna().sum())

Nulls before:  879
Nulls after:  1778


/tmp/ipykernel_519867/4108337641.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df06['publishDate'] = np.where(df06['publishDate'].str.contains(


In [25]:
def convert_date(dft, col):
    """Function to transform date format strings to date data type"""
    date_formats = [
        "%m/%d/%y",
        "%B %d %Y",
        "%B %dth %Y",
        "%B %dst %Y",
        "%B %dnd %Y",
        "%B %drd %Y",
        "%B %Y",
        "%Y"
    ]
 
    for format_str in date_formats:
        dft[col] = pd.to_datetime(dft[col], format=format_str, errors='ignore')

    dft[col] = pd.to_datetime(dft[col], errors='coerce')
    return dft
    

In [26]:
# Clean date format to date data type.

df06['publishDate'] = df06['publishDate'].str.replace(r'\s+(\d(?:st|nd|rd|th)?)\s+', r' 0\1 ', regex=True)

df061 = df06.copy()
df07 = convert_date(df061, 'publishDate')
df08 = convert_date(df07, 'firstPublishDate')

# Correct "future" publishdates. E.g. 06/21/60 is not 2060-06-21. We can determine correct dates between 00-22, but it is a dataset problem.
df08['firstPublishDate'] = np.where(df08['firstPublishDate'].dt.year > 2022,
                                    (df08['firstPublishDate'] - pd.DateOffset(years=100)),
                                    df08['firstPublishDate'])
df08['publishDate'] = np.where(df08['publishDate'].dt.year > 2022,
                               (df08['publishDate'] - pd.DateOffset(years=100)),
                               df08['publishDate'])

print(df08['publishDate'].head(3))
print(df08['firstPublishDate'].head(3))

0   2008-09-14
1   2004-09-28
2   2006-05-23
Name: publishDate, dtype: datetime64[ns]
0          NaT
1   2003-06-21
2   1960-07-11
Name: firstPublishDate, dtype: datetime64[ns]


In [27]:
df08[['publishDate', 'firstPublishDate']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52423 entries, 0 to 52477
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   publishDate       50639 non-null  datetime64[ns]
 1   firstPublishDate  31104 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.2 MB


In [28]:
# Book (title and author) should have unique 'firstPublishDate'
df08.groupby(['title', 'author'])['firstPublishDate'].nunique().sort_values(ascending=False).head(5)


title                        author                                            
The Little House Collection  Laura Ingalls Wilder, Garth Williams (Illustrator)    2
A Song of Ice and Fire       George R.R. Martin                                    2
Eugene Onegin                Alexander Pushkin, James E. Falen (Translator)        2
Oscar and Lucinda            Peter Carey                                           1
Motherless Brooklyn          Jonathan Lethem                                       1
Name: firstPublishDate, dtype: int64

In [29]:
#  Update values for 'firstPublishDate' for 3 inconsistent rows

df08['firstPublishDate'] = np.where(df08['isbn'] =='9780199538645', np.datetime64('1933-10-28'), df08['firstPublishDate'])
df08['firstPublishDate'] = np.where(df08['isbn'] =='9780060529963', np.datetime64('1932-10-30'), df08['firstPublishDate'])
df08['firstPublishDate'] = np.where(df08['isbn'] =='9780345529053', np.datetime64('2000-10-28'), df08['firstPublishDate'])


In [30]:
df08.shape

(52423, 25)

### Clean `pages`

In [31]:
# Pages column should contains integers. Book can not have only 1 page. It is a mistake.
df09 = df08.copy()
df09['pages'] =  df09['pages'].str.replace('1 page', '', regex=True)

## Save clean total dataset

### Prepare list like columns  for loading to postgres

In [32]:
total = df09.copy()
for col in ['genres', 'setting', 'ratingsByStars', 'awards']:
    total[col] = total[col].apply(literal_eval)

In [33]:
total.to_csv('docs/data-preparation_total.csv', index=False) #, lineterminator='\n')

In [34]:
total.columns

Index(['bookId', 'title', 'series', 'author', 'rating', 'description',
       'language', 'isbn', 'genres', 'characters', 'bookFormat', 'edition',
       'pages', 'publisher', 'publishDate', 'firstPublishDate', 'awards',
       'numRatings', 'ratingsByStars', 'likedPercent', 'setting', 'coverImg',
       'bbeScore', 'bbeVotes', 'price'],
      dtype='object')

## Prepare tables for load and save

### 1 . book

In [35]:
# prepare to load characters as a list (postgres array)
book = df09.copy()
to_replace = {'^\[(\'|\")': '{\"',
              '(\'|\")\]$': '\"}',
              '(\'|\"), (\'|\")': '\", \"',
              '\[\]': '',
              ',(\'|\")': '\"'}
# = {'^\[': '{',
 #             '\]$': '}'}

book['characters'] = book['characters'].replace(to_replace, regex=True)
book['characters'].head(2)

0    {"Katniss Everdeen", "Peeta Mellark", "Cato (Hunger Games)", "Primrose Everdeen", "Gale Hawthorne", "Effie Trinket", "Haymitch Abernathy", "Cinna", "President Coriolanus Snow", "Rue", "Flavius", "Lavinia (Hunger Games)", "Marvel", "Glimmer", "Clove", "Foxface", "Thresh", "Greasy Sae", "Madge Und...
1    {"Sirius Black", "Draco Malfoy", "Ron Weasley", "Petunia Dursley", "Vernon Dursley", "Dudley Dursley", "Severus Snape", "Rubeus Hagrid", "Lord Voldemort", "Minerva McGonagall", "Neville Longbottom", "Fred Weasley", "George Weasley", "Percy Weasley", "Ginny Weasley", "Colin Creevey", "Filius Flit...
Name: characters, dtype: object

In [36]:
# book table (title, author, firstPublishDate, characters)
book = book[['title', 'author', 'firstPublishDate', 'characters']]
book = book.drop_duplicates()
book.head(5)

,title,author,firstPublishDate,characters
0,The Hunger Games,Suzanne Collins,NaT,"{""Katniss Everdeen"", ""Peeta Mellark"", ""Cato (Hunger Games)"", ""Primrose Everdeen"", ""Gale Hawthorne"", ""Effie Trinket"", ""Haymitch Abernathy"", ""Cinna"", ""President Coriolanus Snow"", ""Rue"", ""Flavius"", ""Lavinia (Hunger Games)"", ""Marvel"", ""Glimmer"", ""Clove"", ""Foxface"", ""Thresh"", ""Greasy Sae"", ""Madge Und..."
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",2003-06-21,"{""Sirius Black"", ""Draco Malfoy"", ""Ron Weasley"", ""Petunia Dursley"", ""Vernon Dursley"", ""Dudley Dursley"", ""Severus Snape"", ""Rubeus Hagrid"", ""Lord Voldemort"", ""Minerva McGonagall"", ""Neville Longbottom"", ""Fred Weasley"", ""George Weasley"", ""Percy Weasley"", ""Ginny Weasley"", ""Colin Creevey"", ""Filius Flit..."
2,To Kill a Mockingbird,Harper Lee,1960-07-11,"{""Scout Finch"", ""Atticus Finch"", ""Jem Finch"", ""Arthur Radley"", ""Mayella Ewell"", ""Aunt Alexandra"", ""Bob Ewell"", ""Calpurnia (housekeeper)"", ""Tom Robinson"", ""Miss Maudie Atkinson"", ""Judge John Taylor"", ""Dill Harris"", ""Heck Tate"", ""Stephanie Crawford""}"
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",2013-01-28,"{""Mr. Bennet"", ""Mrs. Bennet"", ""Jane Bennet"", ""Elizabeth Bennet"", ""Mary Bennet"", ""Kitty Bennet"", ""Lydia Bennet"", ""Louisa Hurst"", ""Caroline Bingley"", ""Fitzwilliam Darcy"", ""Georgiana Darcy"", ""Lady Catherine de Bourgh"", ""Anne de Bourgh"", ""Colonel Fitzwilliam"", ""Mr. Gardiner"", ""Mrs. Gardiner"", ""Sir W..."
4,Twilight,Stephenie Meyer,2005-10-05,"{""Edward Cullen"", ""Jacob Black"", ""Laurent"", ""Renee"", ""Bella Swan"", ""Billy Black"", ""Esme Cullen"", ""Alice Cullen"", ""Jasper Hale"", ""Carlisle Cullen"", ""Emmett Cullen"", ""Rosalie Hale"", ""Charlie Swan"", ""Mike Newton"", ""Jessica Stanley"", ""Angela Weber"", ""Tyler Crowley""}"


In [37]:
# Function to get the row with maximum length
def get_longest_row(group):
    return group.loc[group['characters'].str.len().idxmax()]


# Grouping by 'title' and 'author', and selecting the genre with maximum length (most info)
book = book.groupby(['title', 'author']).apply(get_longest_row).reset_index(drop=True)


In [38]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52390 entries, 0 to 52389
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             52390 non-null  object        
 1   author            52390 non-null  object        
 2   firstPublishDate  31093 non-null  datetime64[ns]
 3   characters        52390 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.6+ MB


In [39]:
book.to_csv('docs/book.csv', index=False)

### 2. genre

In [40]:
df09.groupby(['title', 'author'])['genres'].count().sort_values(ascending=False)

title                                           author                               
Limits of Destiny                               Sharlyn G. Branson (Goodreads Author)    5
Poderosa                                        Sérgio Klein                             3
The Way of Kings                                Brandon Sanderson (Goodreads Author)     3
The Spaghetti Set: Family Served Italian Style  Rose Marie Boyd (Goodreads Author)       2
Ice Station                                     Matthew Reilly                           2
                                                                                        ..
I, Claudius                                     Robert Graves                            1
I, Claudius/Claudius the God                    Robert Graves                            1
I, Coriander                                    Sally Gardner                            1
I, Cosmo                                        Carlie Sorosiak (Goodreads Author)       1
신의 탑

In [41]:
# table genre (title, author, genres)
genre = df09[['title', 'author', 'genres']]

In [42]:
# deduplicate
genre = genre.drop_duplicates()
genre = genre[genre['genres'] != '[]']

In [43]:
import pandas as pd
from ast import literal_eval

# change to list, remove nulls
genre['genres'] = genre['genres'].apply(literal_eval)

In [44]:
# Function to get the row with maximum length
def get_longest_row(group):
    return group.loc[group['genres'].str.len().idxmax()]

# Grouping by 'title' and 'author', and selecting the genre with maximum length
genre = genre.groupby(['title', 'author']).apply(get_longest_row).reset_index(drop=True)


In [45]:
genre.groupby(['title', 'author'])['genres'].count().sort_values(ascending=False).head(3)

title                                                  author                       
!از قر و قمبیل‌های قلمی بی‌قال و قیل                   بزرگمهر حسین پور                 1
The Book of Psalms: A Translation with Commentary      Robert Alter                     1
The Book of Mormon: Another Testament of Jesus Christ  Joseph Smith Jr. (Translator)    1
Name: genres, dtype: int64

In [46]:
genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47775 entries, 0 to 47774
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   47775 non-null  object
 1   author  47775 non-null  object
 2   genres  47775 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [47]:
# one genre per book line
genre = genre.explode('genres')
genre.head(5)

,title,author,genres
0,!از قر و قمبیل‌های قلمی بی‌قال و قیل,بزرگمهر حسین پور,Comics
0,!از قر و قمبیل‌های قلمی بی‌قال و قیل,بزرگمهر حسین پور,Humor
1,"""A Problem from Hell"": America and the Age of Genocide",Samantha Power,Nonfiction
1,"""A Problem from Hell"": America and the Age of Genocide",Samantha Power,History
1,"""A Problem from Hell"": America and the Age of Genocide",Samantha Power,Politics


In [48]:
genre.to_csv('docs/genre.csv', index=False)

### 3. setting

In [49]:
# setting table (title, author, setting) 
df09.groupby(['title', 'author'])['setting'].count().sort_values(ascending=False)

title                                           author                               
Limits of Destiny                               Sharlyn G. Branson (Goodreads Author)    5
Poderosa                                        Sérgio Klein                             3
The Way of Kings                                Brandon Sanderson (Goodreads Author)     3
The Spaghetti Set: Family Served Italian Style  Rose Marie Boyd (Goodreads Author)       2
Ice Station                                     Matthew Reilly                           2
                                                                                        ..
I, Claudius                                     Robert Graves                            1
I, Claudius/Claudius the God                    Robert Graves                            1
I, Coriander                                    Sally Gardner                            1
I, Cosmo                                        Carlie Sorosiak (Goodreads Author)       1
신의 탑

In [50]:
# deduplicate and remove empty values
setting = df09[['title', 'author', 'setting']]
setting = setting.drop_duplicates()
setting = setting[setting['setting'] != '[]']
setting.head(2)

,title,author,setting
0,The Hunger Games,Suzanne Collins,"['District 12, Panem', 'Capitol, Panem', 'Panem (United States)']"
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)","['Hogwarts School of Witchcraft and Wizardry (United Kingdom)', 'London, England']"


In [51]:
# make it a list
setting['setting'] = setting['setting'].apply(literal_eval)
setting.head(2)

,title,author,setting
0,The Hunger Games,Suzanne Collins,"[District 12, Panem, Capitol, Panem, Panem (United States)]"
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)","[Hogwarts School of Witchcraft and Wizardry (United Kingdom), London, England]"


In [52]:
# Function to get the row with maximum length
def get_longest_row(group):
    return group.loc[group['setting'].str.len().idxmax()]

# Grouping by 'title' and 'author', and selecting the row with maximum length, most info
setting = setting.groupby(['title', 'author']).apply(get_longest_row).reset_index(drop=True)

In [53]:
setting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    11562 non-null  object
 1   author   11562 non-null  object
 2   setting  11562 non-null  object
dtypes: object(3)
memory usage: 271.1+ KB


In [54]:
setting.groupby(['title', 'author'])['setting'].count().sort_values(ascending=False).head(3)

title                                     author                                
"Slowly, Slowly, Slowly," said the Sloth  Eric Carle, Jane Goodall (Foreword by)    1
The Black Arrow                           Robert Louis Stevenson                    1
The Black Circle                          Patrick Carman (Goodreads Author)         1
Name: setting, dtype: int64

In [55]:
# one setting per book line
setting = setting.explode('setting')
setting.head(5)

,title,author,setting
0,"""Slowly, Slowly, Slowly,"" said the Sloth","Eric Carle, Jane Goodall (Foreword by)",Amazon Rainforest
1,#Nerd,Cambria Hebert (Goodreads Author),Maryland (United States)
2,#scandal,Sarah Ockler (Goodreads Author),"Lavender Oaks, Colorado (United States)"
3,'Salem's Lot,Stephen King (Goodreads Author),"Jerusalem's Lot, Maine (United States)"
4,'Til Death,Sharon Sala (Goodreads Author),Kentucky (United States)


In [56]:
setting.to_csv('docs/setting.csv', index=False)

### 4. awards

In [57]:
# award table (title, author, award)
df09.groupby(['isbn'])['awards'].count().sort_values(ascending=False)

isbn
0000195166000    1
9781461107033    1
9781461053743    1
9781461063520    1
9781461065715    1
                ..
9780440415992    1
9780440416432    1
9780440416487    1
9780440416548    1
 978097736462    1
Name: awards, Length: 52423, dtype: int64

In [58]:
#award table (isbn, award)
award = df09[['isbn', 'awards']]
award.info()
award = award[award['awards'] != '[]']
award.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52423 entries, 0 to 52477
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   isbn    52423 non-null  object
 1   awards  52423 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10603 entries, 0 to 52475
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   isbn    10603 non-null  object
 1   awards  10603 non-null  object
dtypes: object(2)
memory usage: 248.5+ KB


In [59]:
# make it a list
award['awards'] = award['awards'].apply(literal_eval)
award.head(2)

,isbn,awards
0,9780439023481,"[Locus Award Nominee for Best Young Adult Book (2009), Georgia Peach Book Award (2009), Buxtehuder Bulle (2009), Golden Duck Award for Young Adult (Hal Clement Award) (2009), Grand Prix de l'Imaginaire Nominee for Roman jeunesse étranger (2010), Books I Loved Best Yearly (BILBY) Awards for Older..."
1,9780439358071,"[Bram Stoker Award for Works for Young Readers (2003), Anthony Award for Young Adult (2004), Mythopoeic Fantasy Award for Children's Literature (2008), Audie Award for Audiobook of the Year (2004), Books I Loved Best Yearly (BILBY) Awards for Older Readers (2004), Colorado Blue Spruce Young Adul..."


In [60]:
# Function to get the row with maximum length
def get_longest_row(group):
    return group.loc[group['awards'].str.len().idxmax()]

# Grouping by 'title' and 'author', and selecting the award with maximum , most info
award = award.groupby(['isbn']).apply(get_longest_row).reset_index(drop=True)

In [61]:
# one award line per book
award = award.explode('awards')
award.head(5)

,isbn,awards
0,0000195170342,Pulitzer Prize for History (2005)
0,0000195170342,Ambassador Book Award for American Studies (2005)
0,0000195170342,Massachusetts Book Award Nominee for Nonfiction (2005)
0,0000195170342,National Book Award Finalist for Nonfiction (2004)
1,0031809014745,ECPA Christian Book Award


In [62]:
# Extract award year information
awards = award.copy()
awards['award'] = awards['awards'].str.extract(r'(.*)\s\(\d{4}\)$')
awards['award_year'] = awards['awards'].str.extract(r'\((\d{4})\)$')
awards['award'] = np.where(awards['award'].isna(), awards['awards'], awards['award'])
awards = awards.drop(columns='awards')
awards[awards['award'].isna()]

,isbn,award,award_year


In [63]:
awards.head(3)

,isbn,award,award_year
0,0000195170342,Pulitzer Prize for History,2005
0,0000195170342,Ambassador Book Award for American Studies,2005
0,0000195170342,Massachusetts Book Award Nominee for Nonfiction,2005


In [64]:
awards.to_csv('docs/award.csv', index=False)

### 5. star

In [65]:
# star table (isbn, ratingsByStars). We have one value por row.
df09.groupby(['isbn'])['ratingsByStars'].count().sort_values(ascending=False)

isbn
0000195166000    1
9781461107033    1
9781461053743    1
9781461063520    1
9781461065715    1
                ..
9780440415992    1
9780440416432    1
9780440416487    1
9780440416548    1
 978097736462    1
Name: ratingsByStars, Length: 52423, dtype: int64

In [66]:
# remove empty values
star = df09[['isbn', 'ratingsByStars']]
print(star.info())
star = star[star['ratingsByStars']!= '[]']
star.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52423 entries, 0 to 52477
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   isbn            52423 non-null  object
 1   ratingsByStars  52423 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 51001 entries, 0 to 52477
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   isbn            51001 non-null  object
 1   ratingsByStars  51001 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [67]:
# make it a list
star['ratingsByStars'] = star['ratingsByStars'].apply(literal_eval)
star.head(2)

,isbn,ratingsByStars
0,9780439023481,"[3444695, 1921313, 745221, 171994, 93557]"
1,9780439358071,"[1593642, 637516, 222366, 39573, 14526]"


In [68]:
# Function to get the row with maximum length
def get_longest_row(group):
    return group.loc[group['ratingsByStars'].str.len().idxmax()]

# Grouping by 'title' and 'author', and selecting the ratingsBySatrs with maximum length, most info
star = star.groupby(['isbn']).apply(get_longest_row).reset_index(drop=True)

In [69]:
# one star line per publication
star = star.explode('ratingsByStars')
star['star']= -(star.groupby('isbn')['ratingsByStars'].cumcount()-5)
star.head(10)

,isbn,ratingsByStars,star
0,0000195166000,74,5
0,0000195166000,92,4
0,0000195166000,80,3
0,0000195166000,19,2
0,0000195166000,7,1
1,0000195170342,6136,5
1,0000195170342,4396,4
1,0000195170342,2255,3
1,0000195170342,638,2
1,0000195170342,286,1


In [70]:
star.to_csv('docs/star.csv', index=False)

### 6. publication table

In [71]:
# publication table (isbn,series,rating,description,language,
#bookFormat,edition,pages,publisher,publishDate,numRatings ,likedPercent, coverImg,
#'bbeScore', 'bbeVotes', 'price')
publication = df09[['isbn', 'bookFormat', 'publisher', 'publishDate', 'description', 'series', 'rating',
                    'language', 'edition', 'pages', 'numRatings', 'likedPercent', 'coverImg',
                    'bbeScore', 'bbeVotes', 'price', 'author', 'title']]
publication.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52423 entries, 0 to 52477
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   isbn          52423 non-null  object        
 1   bookFormat    50950 non-null  object        
 2   publisher     48731 non-null  object        
 3   publishDate   50639 non-null  datetime64[ns]
 4   description   51087 non-null  object        
 5   series        23441 non-null  object        
 6   rating        52423 non-null  float64       
 7   language      48622 non-null  object        
 8   edition       4949 non-null   object        
 9   pages         50080 non-null  object        
 10  numRatings    52423 non-null  int64         
 11  likedPercent  51802 non-null  float64       
 12  coverImg      51818 non-null  object        
 13  bbeScore      52423 non-null  int64         
 14  bbeVotes      52423 non-null  int64         
 15  price         38079 non-null  float6

In [72]:
publication.to_csv('docs/publication.csv', index=False)

In [73]:
publication.columns


Index(['isbn', 'bookFormat', 'publisher', 'publishDate', 'description',
       'series', 'rating', 'language', 'edition', 'pages', 'numRatings',
       'likedPercent', 'coverImg', 'bbeScore', 'bbeVotes', 'price', 'author',
       'title'],
      dtype='object')

#### Please note that `bookId` column was droped and was substituted with `isbn`, `title` and `author`
#### as uniquely identifying rows in this dataset.

#### New columns `award_year` and `star` was added to better distribute available information.
